In [76]:
import torch
import torchvision

n_epochs = 3
batch_size_train = 64
batch_size_test = 1
learning_rate = 0.01
momentum = 0.5
log_interval = 100

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_train, shuffle=True)

test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('./datasets/', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])),
  batch_size=batch_size_test, shuffle=True)

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
      
network = Net()
optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                      momentum=momentum)

train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]

def train(epoch):
  network.train()
  for batch_idx, (data, target) in enumerate(train_loader):
    optimizer.zero_grad()
    output = network(data)
    loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
    if batch_idx % log_interval == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.item()))
      train_losses.append(loss.item())
      train_counter.append(
        (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
      
def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))
  
test()
for epoch in range(1, n_epochs + 1):
  train(epoch)
  test()


Test set: Avg. loss: 2.3316, Accuracy: 1137/10000 (11%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.375533
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.897037
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.210238
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.954134
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.809195
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.697300
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.591065
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.394847
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.804426
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.435383

Test set: Avg. loss: 0.2001, Accuracy: 9406/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.481457


[E thread_pool.cpp:113] Exception in thread pool task: mutex lock failed: Invalid argument

KeyboardInterrupt



In [90]:
a = next(iter(test_loader))
val = network(a[0])
print(F.softmax(val).tolist()[0])
print(F.softmax(val).max(1, keepdims=True), a[1])

[7.614176865899935e-05, 0.00032399053452536464, 0.0017359817866235971, 0.0017552543431520462, 2.4307610146934167e-05, 3.1119805498747155e-05, 3.252402436260127e-08, 0.9930495023727417, 0.00021920475410297513, 0.0027844770811498165]
torch.return_types.max(
values=tensor([[0.9930]], grad_fn=<MaxBackward0>),
indices=tensor([[7]])) tensor([7])


/var/folders/xv/qvm0rjns6kl0ltf5ygwwk0c00000gn/T/ipykernel_26531/1402598666.py:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(val).tolist()[0])
/var/folders/xv/qvm0rjns6kl0ltf5ygwwk0c00000gn/T/ipykernel_26531/1402598666.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(F.softmax(val).max(1, keepdims=True), a[1])
